In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

In [74]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [91]:
data_path = 'niagara.csv'

data = pd.read_csv(data_path, sep=';', header=None, names=['Code', 'Description', 'Lien', 'Detail', 'Flag'], on_bad_lines='skip')
data['DescriptionDetail'] = data['Description'] + " - " + data['Detail']

data.head()

,Code,Description,Lien,Detail,Flag,DescriptionDetail
0,Code,Description,Lien,Detail,Flag,Description - Detail
1,ARC-100,Ajout d’une ligne dans la table tampon de l’ar...,NaN,Cette opération permet d'ajouter une ligne dan...,x,Ajout d’une ligne dans la table tampon de l’ar...
2,ARC-101,Retrait de lignes dans la table tampon de l’ar...,NaN,Ce traitement permet de supprimer des lignes d...,x,Retrait de lignes dans la table tampon de l’ar...
3,ARC-110,Recherche d’éléments dans la table d’archivage...,NaN,Cette opération permet de rechercher des ident...,x,Recherche d’éléments dans la table d’archivage...
4,ARC-120,Suppression d’éléments dans la table d’archiva...,NaN,Cette opération permet de supprimer les identi...,x,Suppression d’éléments dans la table d’archiva...


In [98]:
code_to_description_detail = pd.Series(data.DescriptionDetail.values, index=data.Code).to_dict()


In [99]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [100]:
def extract_code_from_question(question):
    # Implémentez votre logique d'extraction de code ici
    # Ceci est un exemple simplifié
    code = question.split()[-1].rstrip('?')
    return code

In [101]:
def generate_answer(question):
    # Extraire le code à partir de la question posée
    code = extract_code_from_question(question)
    
    # Obtenir la description associée au code, sinon retourner une réponse par défaut
    description = code_to_description.get(code, "Je ne suis pas sûr de la documentation pour ce code.")
    
    # Construire une nouvelle entrée pour le modèle qui intègre à la fois le code et la description
    # pour fournir plus de contexte. Cela peut aider le modèle à générer une réponse plus riche et plus ciblée.
    prompt = f"Code: {code}\nDescription: {description}\nGénérer une explication détaillée basée sur le code et la description fournis:"
    
    # Encoder le prompt pour le modèle
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    # Utiliser le modèle pour générer une sortie basée sur le prompt encodé
    outputs = model.generate(inputs, max_new_tokens=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    
    # Décoder la sortie générée en texte lisible
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Optionnellement, vous pouvez nettoyer la sortie générée pour retirer le prompt initial de la réponse
    clean_response = generated_text.replace(prompt, "").strip()
    
    return clean_response

# Supposons que vous ayez déjà défini `extract_code_from_question` et `code_to_description` précédemment


In [ ]:
question = "Quelle est la fonction de BATCH-FIN-600 ?"
response = generate_answer(question)
print(response)

In [ ]:
code_to_description.get("BATCH-FIN-600")